In [1]:
#mecabで分かち書きしながら、tokenizeする


# mecabによる分かち書き

In [2]:
import json
#コーパスの読み込み
target_path="/data/hatakeyama/python/llm_corpus/corpus_scale_200.jsonl"
lines=[]
with open(target_path, "r") as f:
    for line in f:
        lines.append(json.loads(line)["text"])

#一時ファイルに書き込み
with open("../data/corpus_text.txt","w") as f:
    f.write("")
with open("../data/corpus_text.txt","a") as f:
    for line in lines:
        f.write(line+"\n")

In [3]:
#わかちがき
!mecab -F"%M||||" -E"\n" -b 100000 < ../data/corpus_text.txt  > ../data/corpus_text.txt.tok

input-buffer overflow. The line is split. use -b #SIZE option.
input-buffer overflow. The line is split. use -b #SIZE option.
input-buffer overflow. The line is split. use -b #SIZE option.
input-buffer overflow. The line is split. use -b #SIZE option.
input-buffer overflow. The line is split. use -b #SIZE option.
input-buffer overflow. The line is split. use -b #SIZE option.
input-buffer overflow. The line is split. use -b #SIZE option.
input-buffer overflow. The line is split. use -b #SIZE option.
input-buffer overflow. The line is split. use -b #SIZE option.
input-buffer overflow. The line is split. use -b #SIZE option.
input-buffer overflow. The line is split. use -b #SIZE option.
input-buffer overflow. The line is split. use -b #SIZE option.
input-buffer overflow. The line is split. use -b #SIZE option.
input-buffer overflow. The line is split. use -b #SIZE option.
input-buffer overflow. The line is split. use -b #SIZE option.
input-buffer overflow. The line is split. use -b #SIZE 

In [4]:
len(lines)

1216549

# 学習

In [1]:
from types import SimpleNamespace
import sentencepiece as spm
import sys
import yaml
import os
from special_token_list import *


def yaml_to_namespace(yaml_path):
    with open(yaml_path, 'r') as file:
        # YAMLファイルを辞書として読み込む
        data = yaml.safe_load(file)
        # 辞書をSimpleNamespaceに変換
        return recursive_namespace(data)


def recursive_namespace(data):
    if isinstance(data, dict):
        # 再帰的に辞書の各要素をSimpleNamespaceに変換
        return SimpleNamespace(**{k: recursive_namespace(v) for k, v in data.items()})
    elif isinstance(data, list):
        # リストの要素も変換
        return [recursive_namespace(v) for v in data]
    else:
        # その他のデータ型はそのまま返す
        return data


args = yaml_to_namespace('../30tokenize/config.yaml')

args.input="../data/corpus_text.txt.tok"
#args.vocab_size=3000
1

1

In [2]:
#mecab tokenize
print("begin tokenizing...")

spm.SentencePieceTrainer.train(
        input=args.input,
        model_prefix=args.model_prefix,
        vocab_size=args.vocab_size,
        character_coverage=args.character_coverage,
        model_type=args.model_type,
        num_threads=args.num_threads,
        train_extremely_large_corpus=args.train_extremely_large_corpus,
        user_defined_symbols=[
            BOS_TOKEN,
            EOS_TOKEN,
            PAD_TOKEN,
            CLS_TOKEN,
            SEP_TOKEN,
            EOD_TOKEN,
            MASK_TOKEN,
            NEWLINE_TOKEN,
            EXTRA_TOKEN1,
            EXTRA_TOKEN2,
            EXTRA_TOKEN3,
            EXTRA_TOKEN4,
            EXTRA_TOKEN5,
            EXTRA_TOKEN6,
            EXTRA_TOKEN7,
            EXTRA_TOKEN8,
        ],  # Note: `NEWLINE_TOKEN` is needed in `user_defined_symbols`.
        byte_fallback=True,
        split_digits=True,
        allow_whitespace_only_pieces=True,
        remove_extra_whitespaces=False,
        pretokenization_delimiter="||||",
    )


begin tokenizing...


sentencepiece_trainer.cc(77) LOG(INFO) Starts training with : 
trainer_spec {
  input: ../data/corpus_text.txt.tok
  input_format: 
  model_prefix: tokenizer
  model_type: UNIGRAM
  vocab_size: 65000
  self_test_sample_size: 0
  character_coverage: 0.9995
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 32
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 1
  pretokenization_delimiter: ||||
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 1
  user_defined_symbols: <s>
  user_defined_symbols: </s>
  user_defined_symbols: <pad>
  user_defined_symbols: <CLS>
  user_defined_symbols: <SEP>
  user_defined_symbols: <EOD>
  user_defined_symbols: <MASK>
  user_defined_symbols: 

  user_defined_symbols: <llm-code>
  user_defined_symbols: </llm-code>
  user_defined_symbols: <llm-cod

 <0xB6>
trainer_interface.cc(423) LOG(INFO) Adding meta_piece: <0xB7>
trainer_interface.cc(423) LOG(INFO) Adding meta_piece: <0xB8>
trainer_interface.cc(423) LOG(INFO) Adding meta_piece: <0xB9>
trainer_interface.cc(423) LOG(INFO) Adding meta_piece: <0xBA>
trainer_interface.cc(423) LOG(INFO) Adding meta_piece: <0xBB>
trainer_interface.cc(423) LOG(INFO) Adding meta_piece: <0xBC>
trainer_interface.cc(423) LOG(INFO) Adding meta_piece: <0xBD>
trainer_interface.cc(423) LOG(INFO) Adding meta_piece: <0xBE>
trainer_interface.cc(423) LOG(INFO) Adding meta_piece: <0xBF>
trainer_interface.cc(423) LOG(INFO) Adding meta_piece: <0xC0>
trainer_interface.cc(423) LOG(INFO) Adding meta_piece: <0xC1>
trainer_interface.cc(423) LOG(INFO) Adding meta_piece: <0xC2>
trainer_interface.cc(423) LOG(INFO) Adding meta_piece: <0xC3>
trainer_interface.cc(423) LOG(INFO) Adding meta_piece: <0xC4>
trainer_interface.cc(423) LOG(INFO) Adding meta_piece: <0xC5>
trainer_interface.cc(423) LOG(INFO) Adding meta_piece: <0xC6>


In [2]:
# mecabをしないパターン

args = yaml_to_namespace('../30tokenize/config.yaml')
args.input="/data/hatakeyama/python/llm_corpus/corpus_scale_100.jsonl"
args.input="/data/hatakeyama/python/llm_corpus/corpus_scale_50.jsonl"
print(args.vocab_size)
#mecab tokenize

100000


In [3]:

print("begin tokenizing...")

spm.SentencePieceTrainer.train(
        input=args.input,
        model_prefix=args.model_prefix,
        vocab_size=args.vocab_size,
        character_coverage=args.character_coverage,
        model_type=args.model_type,
        num_threads=args.num_threads,
        train_extremely_large_corpus=args.train_extremely_large_corpus,
        user_defined_symbols=[
            BOS_TOKEN,
            EOS_TOKEN,
            PAD_TOKEN,
            CLS_TOKEN,
            SEP_TOKEN,
            EOD_TOKEN,
            MASK_TOKEN,
            NEWLINE_TOKEN,
            EXTRA_TOKEN1,
            EXTRA_TOKEN2,
            EXTRA_TOKEN3,
            EXTRA_TOKEN4,
            EXTRA_TOKEN5,
            EXTRA_TOKEN6,
            EXTRA_TOKEN7,
            EXTRA_TOKEN8,
        ],  # Note: `NEWLINE_TOKEN` is needed in `user_defined_symbols`.
        byte_fallback=True,
        split_digits=True,
        allow_whitespace_only_pieces=True,
        remove_extra_whitespaces=False,
    )

begin tokenizing...


sentencepiece_trainer.cc(77) LOG(INFO) Starts training with : 
trainer_spec {
  input: /data/hatakeyama/python/llm_corpus/corpus_scale_50.jsonl
  input_format: 
  model_prefix: tokenizer
  model_type: UNIGRAM
  vocab_size: 100000
  self_test_sample_size: 0
  character_coverage: 0.9995
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 32
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 1
  pretokenization_delimiter: 
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 1
  user_defined_symbols: <s>
  user_defined_symbols: </s>
  user_defined_symbols: <pad>
  user_defined_symbols: <CLS>
  user_defined_symbols: <SEP>
  user_defined_symbols: <EOD>
  user_defined_symbols: <MASK>
  user_defined_symbols: 

  user_defined_symbols: <llm-code>
  user_defined_symbols: </llm-code>
  user

る...!みんなぁ!おらに現金を分けてくれぇ!」トランクス「すごい...。島中のゴールドが集まってくr」悟空「きた!総額なんと20万ゴールド!くらえぇぇぇぇぇぇ!」ガバッシャル「うわぁぁぁぁぁぁ▂▅▇█▓▒░('ω')░▒▓█▇▅▂夢かぁぁぁぁぁ!」\nわー!皆が言ってる財布～なるほどー。さすがにシャル財布は使えないけど、リアムは、オシャレ!なんでシャルもそういう感じにしなかったのかな!てかリアムの財布が前買おうか悩んでた感じだ。全然違うけど、もっとレディースでも使いやすい感じなら絶対売れる。"}
trainer_interface.cc(145) LOG(INFO) Loaded 5000000 lines
trainer_interface.cc(122) LOG(WARNING) Too many sentences are loaded! (5295699), which may slow down training.
trainer_interface.cc(124) LOG(WARNING) Consider using --input_sentence_size=<size> and --shuffle_input_sentence=true.
trainer_interface.cc(127) LOG(WARNING) They allow to randomly sample <size> sentences from the entire corpus.
trainer_interface.cc(407) LOG(INFO) Loaded all 5295699 sentences
trainer_interface.cc(414) LOG(INFO) Skipped 1013727 too long sentences.
trainer_interface.cc(423) LOG(INFO) Adding meta_piece: <unk>
trainer_interface.cc(423) LOG(INFO) Adding meta_piece: <s>
trainer_interface.cc(423) LOG(INFO) Adding meta_piece: </s>
trainer_interface.cc(423) LOG(INFO)

# 動作確認

In [1]:

import sentencepiece as spm
#test
model_path="../data/tokenizers/0528tokenizer_100k.model"
model_path="../data/tokenizers/0529tokenizer_100k_wo_mecab.model"
model_path="../data/tokenizers/0529tokenizer_100k_wo_mecab_scale100.model"
model_path="../data/tokenizers/0529tokenizer_100k_wo_mecab_scale50.model"
sp = spm.SentencePieceProcessor(model_file=model_path)


In [3]:

text="""\n価格詳細, 日本の正式代理店で購入する必要がある
he is a good man
import pandas as pd  from tqdm import tqdm

def yaml_to_namespace(yaml_path):
    with open(yaml_path, 'r') as file:
        # YAMLファイルを辞書として読み込む
        data = yaml.safe_load(file)
        # 辞書をSimpleNamespaceに変換
        return recursive_namespace(data)

これはテストです。
吾輩は猫である｡名前はまだない
吾輩は猫である｡\n名前はまだない
1+1=2
1 + 2 = 3
3*4+12.3=24.3
"""
(sp.encode(text, out_type=str)),(sp.encode(text))

(['▁',
  '\n',
  '価格',
  '詳細',
  ',',
  '▁',
  '日本の',
  '正式',
  '代理店',
  'で購入',
  'する必要がある',
  '\n',
  'he',
  '▁is',
  '▁a',
  '▁good',
  '▁man',
  '\n',
  'import',
  '▁pandas',
  '▁as',
  '▁pd',
  '▁',
  '▁from',
  '▁tqdm',
  '▁import',
  '▁tqdm',
  '\n',
  '\n',
  'def',
  '▁yaml',
  '_',
  'to',
  '_',
  'namespace',
  '(',
  'yaml',
  '_',
  'path',
  '):',
  '\n',
  '▁▁▁',
  '▁with',
  '▁open',
  '(',
  'yaml',
  '_',
  'path',
  ',',
  "▁'",
  'r',
  "')",
  '▁as',
  '▁file',
  ':',
  '\n',
  '▁▁▁▁▁▁▁▁',
  '#',
  '▁',
  'YAML',
  'ファイルを',
  '辞書',
  'として',
  '読み込む',
  '\n',
  '▁▁▁▁▁▁▁',
  '▁data',
  '▁=',
  '▁yaml',
  '.',
  'safe',
  '_',
  'load',
  '(',
  'file',
  ')',
  '\n',
  '▁▁▁▁▁▁▁▁',
  '#',
  '▁',
  '辞書',
  'を',
  'Simple',
  'Namespace',
  'に変換',
  '\n',
  '▁▁▁▁▁▁▁',
  '▁return',
  '▁recursive',
  '_',
  'namespace',
  '(',
  'data',
  ')',
  '\n',
  '\n',
  'これは',
  'テスト',
  'です',
  '。',
  '\n',
  '吾',
  '輩',
  'は',
  '猫',
  'である',
  '。',
  '名前は',
  'まだ',
  'ない',
  

In [14]:
t="""あ    """
tokens=sp.encode(t)
t,tokens,sp.decode(tokens)

('あ    ', [276, 1115, 341], 'あ    ')

In [17]:
sp.decode([341])

'   '

In [8]:
#圧縮率の確認
from datasets import load_dataset

def check_compression_ratio(ds,sp):
    text_length=0
    text_tokens=0
    loader=iter(ds)
    for i in range(1000):
        record=next(loader)
        text=(record["text"])
        text_length+=len(text)
        text_tokens+=len(sp.encode(text))
    return text_length/text_tokens



In [9]:

ds=load_dataset("izumi-lab/wikipedia-ja-20230720",split="train",streaming=True)
check_compression_ratio(ds,sp)

2.0822069652981123

In [10]:

ds=load_dataset("wikipedia", "20220301.en", streaming=True)["train"]
check_compression_ratio(ds,sp)

3.8856257809390065